In [1]:
import tushare as ts
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv("future_ss2312_min.csv")


# 1. 数据加载和预处理
data_clean = data.sort_values(by='trade_time')
# 确保'close'列是数值型
data_clean['close'] = pd.to_numeric(data_clean['close'], errors='coerce')
data_clean['vol'] = pd.to_numeric(data_clean['vol'], errors='coerce')

# 2. 特征生成
# Example usage:
rolling_windows = {'mean': 5, 'std': 5, 'rsi': 5}
ewm_spans = {'short': 3, 'long': 10, 'signal': 9}


# Calculate rolling mean and standard deviation
data_clean['rolling_mean'] = data_clean['close'].rolling(window=rolling_windows['mean']).mean()
data_clean['rolling_std'] = data_clean['close'].rolling(window=rolling_windows['std']).std()

# Calculate RSI
delta = data_clean['close'].diff()
gain = (delta.where(delta > 0, 0)).fillna(0)
loss = (-delta.where(delta < 0, 0)).fillna(0)
avg_gain = gain.rolling(window=rolling_windows['rsi']).mean()
avg_loss = loss.rolling(window=rolling_windows['rsi']).mean()
rs = avg_gain / avg_loss
data_clean['RSI'] = 100 - (100 / (1 + rs))

# Calculate MACD
short_ema = data_clean['close'].ewm(span=ewm_spans['short'], adjust=False).mean()
long_ema = data_clean['close'].ewm(span=ewm_spans['long'], adjust=False).mean()
data_clean['MACD'] = short_ema - long_ema
data_clean['MACD_signal'] = data_clean['MACD'].ewm(span=ewm_spans['signal'], adjust=False).mean()

# Shift RSI and MACD to use them as features for next timestep
data_clean['RSI_shifted'] = data_clean['RSI'].shift(1)
data_clean['MACD_shifted'] = data_clean['MACD'].shift(1)
data_clean['MACD_signal_shifted'] = data_clean['MACD_signal'].shift(1)

# Define label
data_clean['label'] = (data_clean['close'].shift(-10) > data_clean['close']).astype(int)

# 3. 分割数据
# 分割数据为训练集和测试集
# Convert the 'trade_time' column to datetime
data_clean['trade_time'] = pd.to_datetime(data_clean['trade_time'])

# Now you can filter the data between two dates
train_data = data_clean[(data_clean['trade_time'] >= '2023-05-01 09:00:00') & 
                        (data_clean['trade_time'] < '2023-10-20 09:00:00')]

test_data = data_clean[data_clean['trade_time'] >= '2023-10-20 09:00:00']


features = ['close','vol', 'rolling_mean', 'rolling_std', 'RSI_shifted', 'MACD_shifted', 'MACD_signal_shifted']
X_train = train_data[features]
y_train = train_data['label']
X_train.dropna(inplace=True)
y_train = y_train[X_train.index]

X_test = test_data[features]
y_test = test_data['label']

# 模型训练
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


/var/folders/g3/764mx2lj2gs3qc7l4pz4v2n00000gp/T/ipykernel_14469/1231437921.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.dropna(inplace=True)


RandomForestClassifier(random_state=42)

In [2]:
def predict_next_move(tick, model, rolling_windows, ewm_spans, historical_data):
    # 将 'last_price' 作为 'current' 进行计算
    tick['current'] = tick['close']
    
    # 将新的 tick 数据追加到历史数据中
    historical_data = pd.concat([historical_data, pd.DataFrame([tick])], ignore_index=True)
    
    # 打印当前已有的数据条数
    # print(f"当前已有的数据条数: {len(historical_data)}")
    
    # 检查我们是否有足够的数据来计算滚动和EWM特征
    if len(historical_data) >= max(rolling_windows['mean'], rolling_windows['std'], rolling_windows['rsi'], ewm_spans['long']):
        # 在历史数据上计算滚动平均和标准差
        historical_data['rolling_mean'] = historical_data['current'].rolling(window=rolling_windows['mean'], min_periods=1).mean()
        historical_data['rolling_std'] = historical_data['current'].rolling(window=rolling_windows['std'], min_periods=1).std()

        # 在历史数据上计算RSI
        delta = historical_data['current'].diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)
        avg_gain = gain.rolling(window=rolling_windows['rsi'], min_periods=1).mean()
        avg_loss = loss.rolling(window=rolling_windows['rsi'], min_periods=1).mean()
        rs = avg_gain / avg_loss.replace(0, 1)  # 避免除以零
        historical_data['RSI'] = 100 - (100 / (1 + rs))

        # 在历史数据上计算MACD
        short_ema = historical_data['current'].ewm(span=ewm_spans['short'], adjust=False, min_periods=1).mean()
        long_ema = historical_data['current'].ewm(span=ewm_spans['long'], adjust=False, min_periods=1).mean()
        historical_data['MACD'] = short_ema - long_ema
        historical_data['MACD_signal'] = historical_data['MACD'].ewm(span=ewm_spans['signal'], adjust=False, min_periods=1).mean()

        # 将RSI和MACD移位以用作下一个时间步的特征
        historical_data['RSI_shifted'] = historical_data['RSI'].shift(1)
        historical_data['MACD_shifted'] = historical_data['MACD'].shift(1)
        historical_data['MACD_signal_shifted'] = historical_data['MACD_signal'].shift(1)

        # 使用最后一行数据进行预测
        X_new = historical_data.iloc[-1:][['close', 'vol','rolling_mean', 'rolling_std', 'RSI_shifted', 'MACD_shifted', 'MACD_signal_shifted']]
        
        # 检查X_new是否包含NaN值
        if X_new.isnull().values.any():
            # 处理包含NaN值的行（例如，跳过预测或使用占位符值）
            # 例如，我们可以返回None或一个特定的信号表示数据不足
            return None, historical_data
        else:
            # 进行预测
            prediction_proba = model.predict_proba(X_new)
            # 获取预测为类别1的概率
            probability_of_one = prediction_proba[0][1]
            return probability_of_one, historical_data
    else:
        # 数据不足以进行预测
        return None, historical_data




# Initialize historical_data with the correct column names and types if necessary
historical_data = pd.DataFrame()

initial_funds = 100000
funds = initial_funds
stock_quantity = 0
stock_price = 0
buy_threshold = 0.8
sold_threshold = 0.3
transactions = []
minute_count = 0

for idx, row in test_data.iterrows():
    current_probability, historical_data = predict_next_move(row, rf, rolling_windows, ewm_spans, historical_data) 
    # print(current_probability)
    # print(row['trade_time'])
    if current_probability is not None:
        current_price = row['close']
        minute_count = minute_count + 1
            
        if current_probability > buy_threshold and stock_quantity == 0 :
            print(current_probability)
            stock_quantity = funds // current_price
            funds -= stock_quantity * current_price
            fee = stock_quantity * 2
            funds -= fee
            stock_price = current_price
            buy_price = current_price  # 记录买入价格
            print(f"Time: {row['trade_time']} - Action: BUY at {current_price}, Quantity: {stock_quantity},Funds:{funds}")
            transactions.append({
                'action': 'buy',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds
            })
            minute_count = 0
        elif minute_count > 10 and stock_quantity > 0:
        # elif current_probability < sold_threshold and stock_quantity > 0 and minute_count>100 or (stock_quantity > 0 and flag==0):
            print(current_probability)
            funds += stock_quantity * current_price
            price_diff = current_price - buy_price  # 计算价格差异
            # 判断价格差异是涨、跌还是平
            if price_diff > 0:
                direction = '涨'
            elif price_diff < 0:
                direction = '跌'
            else:
                direction = '平'
            print(f"Time: {row['trade_time']} - Action: SELL at {current_price}, Quantity: {stock_quantity},Funds:{funds}, Price Change: {direction}")
            transactions.append({
                'action': 'sell',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds,
                'price_change': direction
            })
            stock_quantity = 0
            buy_price = 0  # 重置买入价格为0
        
    
if len(transactions)>0:
    transactions_df = pd.DataFrame(transactions)
    print(transactions_df['price_change'].value_counts())
transactions_df.to_csv('transactions_classify.csv')

0.88
Time: 2023-10-20 09:35:00 - Action: BUY at 14680.0, Quantity: 6.0,Funds:11908.0
0.63
Time: 2023-10-20 09:46:00 - Action: SELL at 14695.0, Quantity: 6.0,Funds:100078.0, Price Change: 涨
0.81
Time: 2023-10-23 23:30:00 - Action: BUY at 14575.0, Quantity: 6.0,Funds:12616.0
0.57
Time: 2023-10-23 23:41:00 - Action: SELL at 14575.0, Quantity: 6.0,Funds:100066.0, Price Change: 平
0.82
Time: 2023-10-24 10:42:00 - Action: BUY at 14610.0, Quantity: 6.0,Funds:12394.0
0.61
Time: 2023-10-24 10:53:00 - Action: SELL at 14610.0, Quantity: 6.0,Funds:100054.0, Price Change: 平
0.81
Time: 2023-10-24 14:32:00 - Action: BUY at 14650.0, Quantity: 6.0,Funds:12142.0
0.76
Time: 2023-10-24 14:43:00 - Action: SELL at 14670.0, Quantity: 6.0,Funds:100162.0, Price Change: 涨
0.82
Time: 2023-10-24 14:59:00 - Action: BUY at 14665.0, Quantity: 6.0,Funds:12160.0
0.47
Time: 2023-10-24 21:09:00 - Action: SELL at 14660.0, Quantity: 6.0,Funds:100120.0, Price Change: 跌
0.87
Time: 2023-10-24 21:56:00 - Action: BUY at 14675.0